#### Import all libraries

In [1]:
from __future__ import print_function
import os
import collections
import SimpleITK as sitk
import numpy as np
import six
import radiomics
from radiomics import firstorder, glcm, imageoperations, shape, glrlm, glszm, getTestCase
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
from pathlib import Path
import sys
import logging as logger
import time
import json
import os
import logging.config
import pandas as pd
import gc
import math

In [2]:
#Read the entire Benthiemer sandstone
global image_1, label_1
filename_ben = 'investigate_bent_8bit_1000cube.nrrd'
image_1 = sitk.ReadImage('D:\\Ankita\\pyradiomics\\data\\investigate_bent_8bit_1000cube.nrrd')

#### Extraction Settings

In [3]:
settings = {}
settings['label'] = 255

In [4]:
%%time
mask_x = 1000
mask_y = 1000
mask_z = 1000

u_window_size = [25]
global window_size, total_iter, expect_iter, i_loc, j_loc, z_loc, bb, correctedMask, croppedImage, croppedMask, glcm_window
for window_size in u_window_size:     
    total_iter = 0
    my_entropy = []
    step_x = window_size-1
    step_y = window_size-1
    step_z = window_size-1
    
    i_loc=0
    j_loc=0
    z_loc=0
    i_loc_max = window_size
    j_loc_max = window_size
    z_loc_max = window_size
    
    while i_loc_max <= mask_x:
        #print("I loop", str(i_loc), str(i_loc_max), str(j_loc), str(j_loc_max), str(z_loc), str(z_loc_max))
        j_loc_max = window_size
        while j_loc_max <= mask_y:
            #print("J loop", str(i_loc), str(i_loc_max), str(j_loc), str(j_loc_max), str(z_loc), str(z_loc_max))
            z_loc_max = window_size
            while z_loc_max <= mask_z:
                #print("Z loop", str(i_loc), str(i_loc_max), str(j_loc), str(j_loc_max), str(z_loc), str(z_loc_max))
                full_mask = np.zeros((mask_x, mask_y, mask_z), dtype = np.uint8)
                full_mask[i_loc:i_loc_max,j_loc:j_loc_max,z_loc:z_loc_max] = 255
                #print(full_mask)
                
                label_1 = sitk.GetImageFromArray(full_mask, isVector=False)
                del full_mask

                bb, correctedMask = imageoperations.checkMask(image_1, label_1, label=255)
                if correctedMask is not None:
                    label_1 = correctedMask
                croppedImage, croppedMask = imageoperations.cropToTumorMask(image_1, label_1, bb)
                del correctedMask, label_1

                poro = my_entropy.append(val)
                del key, val, result, glcmFeatures
                
                z_loc = z_loc + step_z
                total_iter = total_iter + 1
                print(total_iter)
                z_loc_max = z_loc+window_size
            
            z_loc = 0
            j_loc = j_loc + step_y
            j_loc_max = j_loc+window_size
        
        j_loc = 0
        i_loc = i_loc + step_x
        i_loc_max = i_loc+window_size
    
    my_df = pd.DataFrame(my_entropy)
    name_csv = 'entropy_window_' + str(window_size) + '.csv'
    my_df.to_csv(name_csv, index=False, header=False)
    print("Saved glcm values of window size ", str(window_size))
    del my_df, name_csv   
    print("Completed Calculations for Window ", str(window_size), ". The number of iterations is ", str(total_iter))

NameError: name 'np' is not defined

In [3]:
x = [0, 255, 0, 255, 255, 255]

x.count(255)

4

#### Only to be used for window sizes 600, 700, 800, 900

In [ ]:
%%time
mask_x = 1000
mask_y = 1000
mask_z = 1000

u_window_size = [1000, 900, 800, 700, 600, 700, 600] 
global window_size, total_iter, expect_iter, i_loc, j_loc, z_loc, bb, correctedMask, croppedImage, croppedMask, glcm_window
for window_size in u_window_size:     
    total_iter = 0
    my_entropy = []
    
    i_loc=0
    j_loc=0
    z_loc=0
    i_loc_max = window_size
    j_loc_max = window_size
    z_loc_max = window_size
    
    #print("Z loop", str(i_loc), str(i_loc_max), str(j_loc), str(j_loc_max), str(z_loc), str(z_loc_max))
    full_mask = np.zeros((mask_x, mask_y, mask_z), dtype = np.uint8)
    full_mask[i_loc:i_loc_max,j_loc:j_loc_max,z_loc:z_loc_max] = 255
    #print(full_mask)

    label_1 = sitk.GetImageFromArray(full_mask, isVector=False)
    del full_mask

    bb, correctedMask = imageoperations.checkMask(image_1, label_1, label=255)
    if correctedMask is not None:
        label_1 = correctedMask
    croppedImage, croppedMask = imageoperations.cropToTumorMask(image_1, label_1, bb)
    del correctedMask, label_1

    #calculate secondorder stats
    glcmFeatures = glcm.RadiomicsGLCM(croppedImage, croppedMask, **settings)
    glcmFeatures.enableFeatureByName('JointEntropy', True)
    result = glcmFeatures.execute()

    #print('Calculated GLCM features: ')
    for (key, val) in six.iteritems(result):
        my_entropy.append(val)
    del key, val, result, glcmFeatures
    
    my_df = pd.DataFrame(my_entropy)
    name_csv = 'entropy_window_' + str(window_size) + '.csv'
    my_df.to_csv(name_csv, index=False, header=False)
    print("Saved glcm values of window size ", str(window_size))
    del my_df, name_csv   
    print("Completed Calculations for Window ", str(window_size), ". The number of iterations is ", str(total_iter))